In [ ]:
import numpy as np
import os
import time
from tqdm import tqdm
import requests
import pandas as pd
import xlrd
import random

In [ ]:
from utils import *
%load_ext autoreload
%autoreload 2

In [ ]:
sentence_data_path = ""
wordplay_data_list = ""
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

In [ ]:
import openai
openai.api_key = ""
API_ENDPOINT = "https://api.openai.com/v1/chat/completions"

In [ ]:
demonstration = """Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"""
def getSample_demo(sample):
    sample_demo = """
    
Question: {}
Choice:
(A) {}
(B) {}
(C) {}
(D) {}

Answer:""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])
    
    return sample_demo

In [ ]:
test_data_list = sentence_data_list + wordplay_data_list
len(test_data_list)

In [ ]:
data_number = len(test_data_list)
data_index = 0
folder_path = 'log'
category_folder = ''
log_path = category_folder+'_log.txt'
sub_log_path = os.path.join(folder_path,category_folder)
log_txt_path = os.path.join(sub_log_path,log_path)
if not os.path.exists(sub_log_path):
    os.makedirs(sub_log_path) 
if os.path.exists(log_txt_path):
    os.remove(log_txt_path)
    
with tqdm(total=data_number) as pbar: 
    while True:
        try:
            with open(log_txt_path,"a") as file:
                file.write("prompting begin at {data_index}\n".format(data_index=data_index))
            sample = test_data_list[data_index]
            utr_demon = demonstration + getSample_demo(sample)

            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": utr_demon},
                ],
            temperature = 0.0,
            top_p = 0.0,
            )

            response = completion['choices'][0]['message']['content']
            sample['response'] = response
            
            if data_index >0 and data_index%50 ==0:
                np.save(os.path.join(sub_log_path,'prompting_result_{}'.format(data_index)), test_data_list)
            data_index+=1
            pbar.update(1)
            if data_index == data_number:
                break
        except:
            time.sleep(60)
            with open(log_txt_path,"a") as file:
                file.write("reach limitation at {data_index}\n".format(data_index=data_index))
                
                
np.save(os.path.join(folder_path,category_folder,'prompting_result_{}'.format(data_index)), test_data_list)

### filter noise response

In [ ]:
for index,item in enumerate(test_data_list):
    try:
        item['predict'] = item['response'].split('(')[1].strip().split(')')[0].strip()
    except:
        print(index)

In [ ]:
word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play,sentence_play)

# few-shot

In [ ]:
few_shot_examples = list(np.load("../data/demonstration.npy",allow_pickle=True))
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))
test_data_list = sentence_data_list + wordplay_data_list
len(test_data_list)

In [ ]:
few_shot_examples[0]

In [ ]:
def getFewshot_demo(examples,number):
    label_to_answer = ['A','B','C','D']
    few_shot_demon = examples[:number//2]+examples[4:4+number//2]
    fewshot = ''
    for sample in few_shot_demon:
        fewshot += """
        
Question: {}
Choice:
(A) {}
(B) {}
(C) {}
(D) {}

Answer:({}) {}""".format(sample['Question'],sample['choice_list'][0],sample['choice_list'][1]\
                    ,sample['choice_list'][2],sample['choice_list'][3]\
                         ,label_to_answer[int(sample['label'])],sample['choice_list'][int(sample['label'])])   
    
    return fewshot

In [ ]:
demonstration = """Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"""
demon_num = 4
demonstration = demonstration + getFewshot_demo(few_shot_examples,demon_num)

In [ ]:
print(demonstration)

In [ ]:
data_number = len(test_data_list)
data_index = 0
folder_path = 'log'
category_folder = ''
log_path = category_folder+'_log.txt'
sub_log_path = os.path.join(folder_path,category_folder)
log_txt_path = os.path.join(sub_log_path,log_path)
if not os.path.exists(sub_log_path):
    os.makedirs(sub_log_path) 
if os.path.exists(log_txt_path):
    os.remove(log_txt_path)
    
with tqdm(total=data_number) as pbar: 
    while True:
        try:
            with open(log_txt_path,"a") as file:
                file.write("prompting begin at {data_index}\n".format(data_index=data_index))
            sample = test_data_list[data_index]
            utr_demon = demonstration + getSample_demo(sample)

            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": utr_demon},
                ],
            temperature = 0.0,
            top_p = 0.0,
            )

            response = completion['choices'][0]['message']['content']
            sample['response'] = response
            
            if data_index >0 and data_index%50 ==0:
                np.save(os.path.join(sub_log_path,'prompting_result_{}'.format(data_index)), test_data_list)
            data_index+=1
            pbar.update(1)
            if data_index == data_number:
                break
        except:
            time.sleep(60)
            with open(log_txt_path,"a") as file:
                file.write("reach limitation at {data_index}\n".format(data_index=data_index))
                
                
np.save(os.path.join(folder_path,category_folder,'prompting_result_{}'.format(data_index)), test_data_list)

In [ ]:
for index,item in enumerate(test_data_list):
    try:
        item['predict'] = item['response'].split('(')[1].strip().split(')')[0].strip()
    except:
        print(index)

In [ ]:
word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play,sentence_play)